In [21]:
"""
生成器的原理

Python 中函数的工作原理

python 解释器会使用一个叫做 PyEval_EvalFramEx() 去执行 foo 函数
PyEval_EvalFramEx ==> 一个 C 语言的函数
首先会去创建一个栈帧, 从而 python 代码运行在栈帧的上下文, 接着会把 python 代码编译为字节码对象
当 foo 调用子函数 bar 又会创建一个栈帧，然后把控制权交给刚创建的栈帧对象
关键：所有栈帧都是分配在堆内存上（堆内存不是放就会一致存在）
栈帧可以独立于调用者存在
"""
import dis
import inspect

frame = None

def foo():
    bar()
    
def bar():
    global frame
    frame = inspect.currentframe()

foo()
# print(dis.dis(foo))
print(frame.f_code.co_name)
print(frame.f_back.f_code.co_name)  # 调用 bar 的栈帧

bar
foo


In [22]:
def generation_function():
    """
    生成器函数是可以 return 一个 value 的
    """
    name = 'lijh'
    yield 1
    yield name
    return 3
    
# 查看生成器的代码
dis.dis(generation_function)

  5           0 LOAD_CONST               1 ('lijh')
              2 STORE_FAST               0 (name)

  6           4 LOAD_CONST               2 (1)
              6 YIELD_VALUE
              8 POP_TOP

  7          10 LOAD_FAST                0 (name)
             12 YIELD_VALUE
             14 POP_TOP

  8          16 LOAD_CONST               3 (3)
             18 RETURN_VALUE


In [23]:
gen = generation_function()
dis.dis(gen)

  5           0 LOAD_CONST               1 ('lijh')
              2 STORE_FAST               0 (name)

  6           4 LOAD_CONST               2 (1)
              6 YIELD_VALUE
              8 POP_TOP

  7          10 LOAD_FAST                0 (name)
             12 YIELD_VALUE
             14 POP_TOP

  8          16 LOAD_CONST               3 (3)
             18 RETURN_VALUE


In [24]:
"""
生成器对象 gen 还没有执行的时候
f_lasti 等于 -1
f_locals 是空字典
"""
print(gen.gi_frame.f_lasti)
print(gen.gi_frame.f_locals)

-1
{}


In [25]:
"""
调用一次 next 函数
这时候 f_lasti 指向了 6 YIELD_VALUE
实际就是 yield 1 的 Python 代码

这样一来：就知道了对于一个函数执行的暂停和执行的理论基础
通过这个例子可以知道一个函数运行到了哪里， 在下一次可以继续运行

而且生成器函数返回的是一个 生成器对象
每个被函数返回的生成器对象都有一个 PyGenObjecet 的实例
所以每个生成器对象之间相互隔离，各自都有自己的指向函数代码的指针
以及各自都保存了独立的运行的上下文环境

生成器对象 也是存储在 堆内存中的
"""
print(next(gen))
print(gen.gi_frame.f_lasti)
print(gen.gi_frame.f_locals)

1
6
{'name': 'lijh'}


In [26]:
print(next(gen))
print(gen.gi_frame.f_lasti)
print(gen.gi_frame.f_locals)

lijh
12
{'name': 'lijh'}
